In [16]:
import yfinance as yf
import datetime
import pandas as pd
import plotly.graph_objects as go
import math
import plotly.io as pio
import numpy as np
import pickle
import os
TODAY = datetime.datetime.today()
# TODAY = (TODAY-datetime.timedelta(days=2))
print(TODAY)

2021-12-08 21:34:04.274310


In [17]:
def get_nifty50_stocks():
    #PROCESS AND LOAD NIFTY 50 DATA
    nif50 = pd.read_html('https://www.moneyseth.com/blogs/Nifty-50-Stock-List-and-its-Weightage-in-Index')[1]
    new_header = nif50.iloc[0] #grab the first row for the header
    nif50 = nif50[1:] #take the data less the header row
    nif50.columns = new_header #set the header row as the df header
    nif50.drop(columns='Sl No', axis=1, inplace=True)
    return nif50

def get_data_date(SYM, START=None, END=None):
    if SYM[0]=='^' or '=' in SYM:
        NAME = f'{SYM}'
    else:
        NAME = f'{SYM}.NS'
    
    df = yf.download(NAME,
                 start=START,
                 end=END,
                 progress=False)
    df.reset_index(inplace=True)

    #RESISTANCE/SUPPORT FINDING
    df['Topline'] = df.apply(lambda x : x.Open if x.Open>x.Close else x.Close, axis=1)
    df['Bottomline'] = df.apply(lambda x : x.Open if x.Open<x.Close else x.Close, axis=1)

    #Create 200 period Moving Average
    df['Moving_Average_200'] = df.Close.rolling(window=200).mean().shift(1)
    df['Moving_Average_50'] = df.Close.rolling(window=50).mean().shift(1)
    df['Moving_Average_20'] = df.Close.rolling(window=20).mean().shift(1)
    
    df['ATR1'] = df.High-df.Low
    df['ATR2'] = abs(df.High-df.Close.shift(-1))
    df['ATR3'] = abs(df.Low-df.Close.shift(-1))
    
    
    df['ATR'] = df[['ATR1','ATR2', 'ATR3']].max(axis=1) # USE to get level area range
    df['200_Avg_ATR'] = df.ATR.rolling(window=200).mean()
    df.drop(columns=['ATR1','ATR2', 'ATR3'], inplace=True)
    
    return df
WIN_RATE = pd.read_csv('WinRate.csv')
def get_win_rate(df):
    global WIN_RATE
    df['Up_Probability'] = df.apply(lambda x :  WIN_RATE[(WIN_RATE.Trend==x.Trend)&(WIN_RATE.Trigger_Type==x.Trigger_Type)].Up_Probability.mean(),axis=1)
    return df

In [18]:
def detect_doji(o,h,l,c):
    body = abs(o-c)
    rng  = abs(h-l)
    
    if body/rng <=0.25:
        return True
    return False

def detect_bullish_engulfing(po,pc,o,c):
    if po>pc and o<c and c>po and o < pc:
        return True
    return False

def detect_bearish_engulfing(po,pc,o,c):
    if po<pc and o>c and c<po:
        return True
    return False

In [19]:
def get_rsi(df, cols=None, n=14):
    df['change'] = (df.Close - df.Close.shift(1))/df.Close.shift(1)
    df['up_change'] = df.change.apply(lambda x: x if x>0 else 0)
    df['up_mean_change'] = df.up_change.rolling(n).mean()
    df['down_change'] = df.change.apply(lambda x: x if x<0 else 0)
    df['down_mean_change'] = df.down_change.rolling(n).mean()
    df[f'rsi_{n}'] = 100-(100/(1+(df.up_mean_change/(-df.down_mean_change))))
    if cols is None:
        df.drop(columns=['up_change','up_mean_change','down_change','down_mean_change'], inplace=True)
        return df
    return df[cols]

def get_resistance_level(df):
    df['Topline-1'] = df.Topline - df.Topline.shift(-1)
    df['Topline-2'] = df.Topline - df.Topline.shift(-2)
    df['Topline-3'] = df.Topline - df.Topline.shift(-3)
    df['Topline-4'] = df.Topline - df.Topline.shift(-4)
    df['Topline+1'] = df.Topline - df.Topline.shift(1)
    df['Topline+2'] = df.Topline - df.Topline.shift(2)
    df['Topline+3'] = df.Topline - df.Topline.shift(3)
    df['Topline+4'] = df.Topline - df.Topline.shift(4)
    
    df['is_Resistance'] = (df['Topline-1']>=0)&(df['Topline-2']>=0)&(df['Topline+1']>=0)&(df['Topline+2']>=0)
    df['is_Resistance_strong'] = (df['is_Resistance'])&(df['Topline-3']>=0)&(df['Topline-4']>=0)&(df['Topline+3']>=0)&(df['Topline+4']>=0)
    df.drop(columns= ['Topline-1','Topline-2','Topline-3','Topline-4','Topline+1','Topline+2','Topline+3','Topline+4'], inplace=True)
    return df

def get_support_level(df):
    df['Bottomline-1'] = df.Bottomline - df.Bottomline.shift(-1)
    df['Bottomline-2'] = df.Bottomline - df.Bottomline.shift(-2)
    df['Bottomline-3'] = df.Bottomline - df.Bottomline.shift(-3)
    df['Bottomline-4'] = df.Bottomline - df.Bottomline.shift(-4)
    df['Bottomline+1'] = df.Bottomline - df.Bottomline.shift(1)
    df['Bottomline+2'] = df.Bottomline - df.Bottomline.shift(2)
    df['Bottomline+3'] = df.Bottomline - df.Bottomline.shift(3)
    df['Bottomline+4'] = df.Bottomline - df.Bottomline.shift(4)
    
    df['is_Support'] = (df['Bottomline-1']<=0)&(df['Bottomline-2']<=0)&(df['Bottomline+1']<=0)&(df['Bottomline+2']<=0)
    df['is_Support_strong'] = (df['is_Support'])&(df['Bottomline-3']<=0)&(df['Bottomline-4']<=0)&(df['Bottomline+3']<=0)&(df['Bottomline+4']<=0)
    df.drop(columns= ['Bottomline-1','Bottomline-2','Bottomline-3','Bottomline-4','Bottomline+1','Bottomline+2','Bottomline+3','Bottomline+4'], inplace=True)
    return df

#THIS NEEDS TO BE REFINED AND TESTED
def get_trend(df): 
    def get_strength(cng):
        cng = abs(cng)
        if cng < 0.12 :
            return 'Weak '
        elif cng <= 0.3:
            return 'Healthy '
        elif cng > 0.3:
            return 'Strong '
        else:
            return '-'
    df['cng'] = (df.Close - df.Close.shift(250))/df.Close.shift(250)
    df['Trend'] = df['cng'].apply(lambda x : get_strength(x)+('Up' if x>0 else ('Down' if x<0 else '-')))
    df.drop(columns=['cng'], inplace=True)
    return df

def which_level_support_probable(df):
    cp = df.iloc[-1].Close
    try:
        support_1 = df[(df.is_Support)&(df.Bottomline<cp)].iloc[-1]['Bottomline']
    except:
        return ('Not Sure', None)
    try:
        support_2 = df[(df.is_Support)&(df.Bottomline<cp)].iloc[-2]['Bottomline']
    except:
        return ('Not Sure', support_1)
    
    
    if abs(support_1-support_2) <= df[df.is_Support].iloc[-1]['ATR']:
        return ('Support_Level', support_1)
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_20']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_20']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_20']<cp):
        return ('Moving_Average_20', df.iloc[-1]['Moving_Average_20'])
    
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_50']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_50']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_50']<cp):
        return ('Moving_Average_50', df.iloc[-1]['Moving_Average_50'])
    
    if (abs(support_1-df[df.is_Support].iloc[-1]['Moving_Average_200']) <= df[df.is_Support].iloc[-1]['ATR'] and
       abs(support_2-df[df.is_Support].iloc[-2]['Moving_Average_200']) <= df[df.is_Support].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_200']<cp):
        return ('Moving_Average_200', df.iloc[-1]['Moving_Average_200'])
    
    return ('Not Sure', support_1)

def which_level_resistance_probable(df):
    cp = df.iloc[-1].Close
    try:
        resistance_1 = df[(df.is_Resistance)&(cp<df.Topline)].iloc[-1]['Topline']
    except:
        return ('Not Sure', None)
    try:
        resistance_2 = df[(df.is_Resistance)&(cp<df.Topline)].iloc[-2]['Topline']
    except:
        return ('Not Sure', resistance_1)
    
    
    if abs(resistance_1-resistance_2) <= df[df.is_Resistance].iloc[-1]['ATR']:
        return ('Resistance_Level', resistance_1)
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_20']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_20']) <= df[df.is_Resistance].iloc[-2]['ATR'] and 
       df.iloc[-1]['Moving_Average_20']>cp):
        return ('Moving_Average_20', df.iloc[-1]['Moving_Average_20'])
    
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_50']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_50']) <= df[df.is_Resistance].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_50']>cp):
        return ('Moving_Average_50', df.iloc[-1]['Moving_Average_50'])
    
    if (abs(resistance_1-df[df.is_Resistance].iloc[-1]['Moving_Average_200']) <= df[df.is_Resistance].iloc[-1]['ATR'] and
       abs(resistance_2-df[df.is_Resistance].iloc[-2]['Moving_Average_200']) <= df[df.is_Resistance].iloc[-2]['ATR'] and
       df.iloc[-1]['Moving_Average_200']>cp):
        return ('Moving_Average_200', df.iloc[-1]['Moving_Average_200'])
    
    return ('Not Sure', resistance_1)


In [20]:
def detect_area_of_value(cp, ATR, sup_level, res_level, trend, sup_type='Not Sure', res_type='Not Sure'):
    if 'Up' in trend:
#         if sup_type=='Not Sure':
#             return False
        try:
            sr_range = abs(sup_level-res_level)
        except:
            sr_range = 0
        if ((sup_level+ATR) >= cp >= (sup_level-ATR)) and cp<=(sup_level+sr_range/2):
            return True
        
    if 'Down' in trend:
        if res_type=='Not Sure':
            return False
        try:
            sr_range = abs(sup_level-res_level)
        except:
            sr_range = 0
        if ((res_level-ATR) <= cp >= (res_level+ATR)) and cp>=(res_level-sr_range/2):
            return True
    return False


def detect_trigger(df):
    if 'Up' in df.iloc[-1].Trend:
        if detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            if min(df.iloc[-1].Open, df.iloc[-1].Close)>=((df.iloc[-1].High+df.iloc[-1].Low)/2):
                return (True, 'Up_Doji')
        if detect_bullish_engulfing(df.iloc[-2].Open, df.iloc[-2].Close, df.iloc[-1].Open, df.iloc[-1].Close):
            return (True, 'Bullish_Engulfing')
        if (df.iloc[-2].rsi_14 < 30) and (df.iloc[-2].rsi_14 < df.iloc[-1].rsi_14) and not detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            return (True, 'RSI')
        
    if 'Down' in df.iloc[-1].Trend:
        if detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            if max(df.iloc[-1].Open, df.iloc[-1].Close)<=((df.iloc[-1].High+df.iloc[-1].Low)/3):
                return (True, 'Down_Doji')
        if detect_bearish_engulfing(df.iloc[-2].Open, df.iloc[-2].Close, df.iloc[-1].Open, df.iloc[-1].Close):
            return (True, 'Bearish_Engulfing')
        if (df.iloc[-2].rsi_14 > 80) and (df.iloc[-2].rsi_14 > df.iloc[-1].rsi_14) and not detect_doji(df.iloc[-1].Open, df.iloc[-1].High, df.iloc[-1].Low, df.iloc[-1].Close):
            return (True, 'RSI')
        
    return (False, None)

In [21]:
def process_df(df):
    df = get_resistance_level(df)
    df = get_support_level(df)
    df = get_trend(df)
    df = get_rsi(df)
    cp = df.iloc[-1].Close
    ATR = df.iloc[-1].ATR
    trend = df.iloc[-1].Trend
    MA20 = df.iloc[-1].Moving_Average_20
    MA50 = df.iloc[-1].Moving_Average_50
    MA200 = df.iloc[-1].Moving_Average_200
    sup_type, sup_level = which_level_support_probable(df)
    res_type, res_level = which_level_resistance_probable(df)
    is_in_area_of_value = detect_area_of_value(cp, ATR, sup_level, res_level, trend, sup_type, res_type)
    is_trigger, trigger_type = detect_trigger(df)
    
    return {'Close' : cp, 'ATR' : ATR, 'Trend' : trend, 'Support_Type' : sup_type, 'Support_Level' : sup_level,
            'Resistance_Type' : res_type, 'Resistance_Level' : res_level, 'In_Area_of_Value' : is_in_area_of_value,
            'Is_Trigger' : is_trigger, 'Trigger_Type' : trigger_type, 'Moving_Average_20':MA20, 'Moving_Average_50':MA50,
            'Moving_Average_200':MA200}

In [22]:
def get_for_bt(d,t):
    df = get_data_date(t,START=datetime.datetime.strptime(d,'%Y-%m-%d')-datetime.timedelta(days=450),END=datetime.datetime.strptime(d,'%Y-%m-%d')+datetime.timedelta(days=1)) #REMOVE TODAY.date() as END
    row = process_df(df)
    row['NAME'] = t
    row['Date'] = df.iloc[-1].Date
    row['Action'] = ('Buy' if ('Up' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else(
                        'Sell' if ('Down' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else None))
    row['Target'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]-0.2*row['ATR'] if row['Action']=='Buy' else(
                     row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]+0.2*row['ATR'] if row['Action']=='Sell' else None))
    
    row['Stoploss'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]+row['ATR'] if row['Action']=='Sell' else(
                         row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]-row['ATR'] if row['Action']=='Buy' else None))
    try:
        row['Expected_Within'] = int((abs(row['Close']-row['Target'])/((df.iloc[-1].ATR+df.iloc[-2].ATR+df.iloc[-3].ATR)/3))+1)
    except:
        row['Expected_Within'] = None
    try:  
        row['Risk_to_Reward'] = round((row['Close']-row['Stoploss'])/(row['Target']-row['Close']), 2)
    except:
        row['Risk_to_Reward'] = None
        
    return pd.DataFrame([row])

In [23]:
nif50 = get_nifty50_stocks()['NSE Symbol'].to_list()
nif50.extend(['^NSEI',"INR=X",'MCDOWELL-N'])

final = []
c = 0
t = len(nif50)
for eq in nif50:
    c += 1
    print(f'{c} out of {t} | Going for {eq}               ', end='\r')
    df = get_data_date(eq,str((TODAY-datetime.timedelta(days=450)).date()),str((TODAY+datetime.timedelta(days=1)).date())) #REMOVE TODAY.date() as END
    row = process_df(df)
    row['NAME'] = eq
    row['Date'] = df.iloc[-1].Date
    row['Action'] = ('Buy' if ('Up' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else(
                        'Sell' if ('Down' in row['Trend'] and row['In_Area_of_Value'] and row['Is_Trigger']) else None))
    row['Target'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]-0.2*row['ATR'] if row['Action']=='Buy' else(
                     row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]+0.2*row['ATR'] if row['Action']=='Sell' else None))
    
    row['Stoploss'] = (row['Resistance_Level' if row['Resistance_Type']=='Not Sure' else row['Resistance_Type']]+row['ATR'] if row['Action']=='Sell' else(
                         row['Support_Level' if row['Support_Type']=='Not Sure' else row['Support_Type']]-row['ATR'] if row['Action']=='Buy' else None))
    try:
        row['Expected_Within'] = int((abs(row['Close']-row['Target'])/((df.iloc[-1].ATR+df.iloc[-2].ATR+df.iloc[-3].ATR)/3))+1)
    except:
        row['Expected_Within'] = None
    try:  
        row['Risk_to_Reward'] = round((row['Close']-row['Stoploss'])/(row['Target']-row['Close']), 2)
    except:
        row['Risk_to_Reward'] = None
    final.append(row)
    
df = pd.DataFrame(final)

In [24]:
from flask import Flask, render_template, jsonify, request

app = Flask(__name__)
cols = ['Date','NAME', 'Support_Level', 'Resistance_Level','Action','Close','Target','Stoploss','Expected_Within','Risk_to_Reward']

@app.route('/home')
def home():
    return render_template('index_template.html', RECOM_TABLE='', HEADER= render_template('header_home.html'))

@app.route('/recommendations')
def recommendations():
    global df
    dfs = df[df.Action.notnull()][cols]
    for i in ['Support_Level','Target','Stoploss','Resistance_Level','Expected_Within','Close']:
        dfs[i] = dfs[i].astype(int)
    s = dfs.style.format().hide_index()
    s = s.set_table_attributes('class="w3-table w3-hoverable"').render()
    return render_template('index_template.html', RECOM_TABLE=s, 
                           HEADER= render_template('header_recommendations.html'))

@app.route('/backtest')
def backtest():
    t = request.args.get('Ticker')
    d = request.args.get('Date')
    dfs = get_for_bt(d,t)
#     print(dfs)
    dfs = dfs[cols]
#     for i in ['Support_Level','Target','Stoploss','Resistance_Level','Expected_Within','Close']:
#         dfs[i] = dfs[i].astype(int)
    s = dfs.style.format().hide_index()
    s = s.set_table_attributes('class="w3-table w3-hoverable"').render()
    return render_template('index_template.html', RECOM_TABLE=s, HEADER= render_template('header_backtest.html'))

app.run(host='0.0.0.0', debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://172.19.59.153:5000/ (Press CTRL+C to quit)


In [2]:
WIN_RATE = pd.read_csv('WinRate.csv')

In [11]:
WIN_RATE['PC'] = WIN_RATE.apply(lambda x : WIN_RATE[(WIN_RATE.Trend==x.Trend)&(WIN_RATE.Trigger_Type==x.Trigger_Type)&(WIN_RATE.Dir=='+')]['count'].mean(), axis=1)
WIN_RATE['NC'] = WIN_RATE.apply(lambda x : WIN_RATE[(WIN_RATE.Trend==x.Trend)&(WIN_RATE.Trigger_Type==x.Trigger_Type)&(WIN_RATE.Dir=='-')]['count'].mean(), axis=1)
WIN_RATE['Up_Probability'] = WIN_RATE['PC']/(WIN_RATE['PC']+WIN_RATE['NC'])
WIN_RATE['Up_Probability'] = WIN_RATE.apply(lambda x : x.Up_Probability if (x.PC+x.NC)>=10 else None,axis=1)



In [13]:
WIN_RATE.to_csv('WinRate.csv',index=False)

In [26]:
get_win_rate(df)

,Close,ATR,Trend,Support_Type,Support_Level,Resistance_Type,Resistance_Level,In_Area_of_Value,Is_Trigger,Trigger_Type,...,Moving_Average_50,Moving_Average_200,NAME,Date,Action,Target,Stoploss,Expected_Within,Risk_to_Reward,Up_Probability
0,2418.100098,25.500000,Healthy Up,Support_Level,2362.600098,Resistance_Level,2498.399902,False,False,None,...,2539.135005,2219.826003,RELIANCE,2021-12-08,None,None,None,None,None,NaN
1,1553.800049,21.050049,Healthy Up,Support_Level,1503.800049,Moving_Average_50,1585.131006,False,False,None,...,1585.131006,1518.700001,HDFCBANK,2021-12-08,None,None,None,None,None,NaN
2,1753.349976,27.200073,Strong Up,Support_Level,1695.300049,Resistance_Level,1771.099976,False,False,None,...,1723.555007,1544.555257,INFY,2021-12-08,None,None,None,None,None,NaN
3,753.400024,14.700012,Strong Up,Support_Level,709.549988,Resistance_Level,768.299988,False,False,None,...,747.910001,669.085000,ICICIBANK,2021-12-08,None,None,None,None,None,NaN
4,2853.000000,43.949951,Healthy Up,Not Sure,2665.000000,Resistance_Level,3004.000000,False,False,None,...,2836.770996,2642.563995,HDFC,2021-12-08,None,None,None,None,None,NaN
5,3626.899902,45.449951,Strong Up,Support_Level,3536.399902,Resistance_Level,3646.449951,False,False,None,...,3593.801978,3354.211498,TCS,2021-12-08,None,None,None,None,None,NaN
6,1920.449951,54.550049,Weak Up,Not Sure,1885.199951,Resistance_Level,2023.000000,True,False,None,...,2030.932996,1850.764746,KOTAKBANK,2021-12-08,None,None,None,None,None,NaN
7,1825.050049,39.000000,Strong Up,Support_Level,1782.650024,Not Sure,1961.250000,False,True,RSI,...,1809.929001,1586.392999,LT,2021-12-08,None,None,None,None,None,0.769231
8,2340.600098,26.199951,Weak Up,Support_Level,2306.500000,Resistance_Level,2383.300049,False,True,Bullish_Engulfing,...,2488.908999,2436.983247,HINDUNILVR,2021-12-08,None,None,None,None,None,NaN
9,224.949997,4.649994,Healthy Up,Support_Level,217.899994,Moving_Average_20,228.437498,False,False,None,...,233.400999,215.895000,ITC,2021-12-08,None,None,None,None,None,NaN
